연	월	일	시	건물번호	기온(C)	강수량(mm)	풍속(m/s)	습도(%)
 - XGBRegressor: 0.9655916940702893
 - Dacon 점수: 17.44154

연	월	일	시	건물번호	기온(C)	강수량(mm)	풍속(m/s)	습도(%) 

In [1]:
import pandas as pd

In [2]:
building_info = pd.read_csv('./data/building_info.csv')
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')
sample = pd.read_csv('./data/sample_submission.csv')
train.head(3)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88


In [3]:
building_info.replace('-', 0, inplace=True)
building_info.head(3)

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.0,0,0,0
1,2,건물기타,122233.47,99000.0,0,0,0
2,3,건물기타,171243.00,113950.0,40,0,0


In [4]:
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(sparse_output=False)
building_type = onehotencoder.fit_transform(building_info[['건물유형']])

temp = pd.DataFrame(building_type)
temp.columns = onehotencoder.categories_
temp = temp.astype('int32')
temp.head(3)
# print(onehotencoder.categories_)
# print(building_type.shape)
# building_type

,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
building_info['건물유형'].unique()

array(['건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소',
       '지식산업센터', '할인마트', '호텔및리조트'], dtype=object)

In [6]:
test.head(3)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75


In [7]:
def data_preprocessing(data):  
    data.fillna(0, inplace=True)
    data['연'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][:4]))
    data['월'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][4:6]))
    data['일'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][6:8]))
    data['시'] = data['num_date_time'].apply(lambda x: int(x.split(' ')[1]))
    del data['일시']
    del data['num_date_time']
    display(data.head(3))
    return data
    
train = data_preprocessing(train)

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),연,월,일,시
0,1,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,2022,6,1,0
1,1,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,2022,6,1,1
2,1,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2022,6,1,2


In [8]:
train.groupby('건물번호').mean()

,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),연,월,일,시
건물번호,,,,,,,,,,,
1,25.786716,0.565539,2.324314,77.027941,0.173284,0.616608,2653.021412,2022.0,6.929412,14.835294,11.5
2,25.818529,0.495441,2.342353,76.972059,0.173284,0.616863,1677.016676,2022.0,6.929412,14.835294,11.5
3,25.786716,0.565539,2.324314,77.027941,0.173284,0.616608,1444.078500,2022.0,6.929412,14.835294,11.5
4,25.786716,0.565539,2.324314,77.027941,0.173284,0.616608,995.425985,2022.0,6.929412,14.835294,11.5
5,25.786716,0.565539,2.324314,77.027941,0.173284,0.616608,2739.049588,2022.0,6.929412,14.835294,11.5
...,...,...,...,...,...,...,...,...,...,...,...
96,24.812745,0.222402,3.657500,83.896569,0.237892,0.769397,2808.038088,2022.0,6.929412,14.835294,11.5
97,27.263186,0.203775,3.229804,78.709755,0.308186,0.836265,1235.922176,2022.0,6.929412,14.835294,11.5
98,25.152696,0.181569,1.916078,79.894559,0.262451,0.000000,1136.114912,2022.0,6.929412,14.835294,11.5


In [9]:
columns = '연 월 일 시 건물번호 기온(C) 강수량(mm) 풍속(m/s) 습도(%)'.split()
data = train[columns]
y = train['전력소비량(kWh)']
data.head(1)

,연,월,일,시,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%)
0,2022,6,1,0,1,18.6,0.0,0.9,42.0


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.25, random_state=987)


In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

XR = XGBRegressor()
XR.fit(X_train, y_train)
y_pred = XR.predict(X_test)
r2_score(y_test, y_pred)


0.9655916940702893

In [12]:
test_data = data_preprocessing(test)
test_data = test_data[columns]
test_data.head(3)

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),연,월,일,시
0,1,23.5,0.0,2.2,72,2022,8,25,0
1,1,23.0,0.0,0.9,72,2022,8,25,1
2,1,22.7,0.0,1.5,75,2022,8,25,2


,연,월,일,시,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%)
0,2022,8,25,0,1,23.5,0.0,2.2,72
1,2022,8,25,1,1,23.0,0.0,0.9,72
2,2022,8,25,2,1,22.7,0.0,1.5,75


In [13]:
answer = XR.predict(test_data)
answer

array([1984.8647 , 1963.7201 , 1884.4098 , ...,  715.0567 ,  618.64276,
        496.988  ], dtype=float32)

In [14]:
submission = pd.DataFrame()
submission['num_date_time'] = sample['num_date_time']
submission['answer'] = answer
submission.head()

,num_date_time,answer
0,1_20220825 00,1984.864746
1,1_20220825 01,1963.720093
2,1_20220825 02,1884.409790
3,1_20220825 03,1938.069580
4,1_20220825 04,1925.296143


In [15]:
import datetime

now = datetime.datetime.now().strftime("%Y-%m-%d %H시%M분%S초")
# submission.to_csv(f'{now}.csv', index=False)